<a href="https://colab.research.google.com/github/coderTanisha22/Jarvis_learnings/blob/main/jarvis_STT%2C_RAG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPUs are preffered for
1. Speech Recognition (Voice input)	Heavy models, real-time audio

	whisper, whisper-large
2. Large Language Models (LLMs)	Need fast inference + more memory
llama, mistral, phi-2
3. Document summarization	Long context input, Transformer-heavy

	bart-large-cnn, t5-xxl
4. Image understanding (OCR/Vision)	Vision transformers can be large

	CLIP, Donut, BLIP
5. Real-time multimodal Jarvis	Voice + Text + Image combo
Whisper + GPT + TTS

Disadvanntage of using colab
Everyting is lost when session expires, the installed packages, variables etc.

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="gpt2", device=0)  # 0 = use first GPU
sentiment = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
sentiment("I love Jarvis!")

In [ ]:
sentiment("I love Jarvis!")

In [ ]:
pipe("I love coding")

In [ ]:
#from transformers import pipeline
#translating english to french
translator = pipeline("translation_en_to_fr", model="t5-base")
translator("Hello, how are you?")

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from pydub import AudioSegment

# Use the uploaded file name exactly
audio = AudioSegment.from_file("er voice 2.m4a")
audio.export("audio.wav", format="wav")


converting speech to text LLM

In [ ]:
#converting speech to text
#from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    return_timestamps=True
)

result = pipe("audio.wav")
print(result['text'])

Q/A LLM

In [ ]:
qa = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")


In [ ]:
context = """
India is the world's largest democracy. It is located in South Asia and is bordered by Pakistan, China, Nepal, Bhutan, Bangladesh, and Myanmar.
It has a population of over 1.4 billion people and a parliamentary system of government.
"""


In [ ]:
question = "Which countries border India?"
result = qa(question=question, context=context)
print("Answer:", result['answer'])


In [ ]:
ques = "Population of India?"
result = qa(question=ques, context=context)
print("Answer:", result['answer'])

Code Generation LLMs

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model
model_id = "Salesforce/codegen-350M-multi" #model name, codegen for supporting multiple programming language
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

prompt = """Write a standalone Python function that prints factorial of n."""

#tokenizing the prompt and returning to pytorch
inputs = tokenizer(prompt, return_tensors="pt")

# Generate code
outputs = model.generate(
    inputs.input_ids, #generating the tokens of input taken
    max_length=200,  #maximum length of output, else it may run slow
    num_return_sequences=1,  #to get single or variations of code
    do_sample=True, #we do this for random, creative code generation (not only greedy)
    temperature=0.7, #randomness level, 0.7 is balanced
)

# Decode the tokens
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
#output[0] refers to the first set of generated output, here we have generated only one output
#skip for skipping some special tokens which are added during the conversion like <pad> , <s>, </s>
print(generated_code)


Working on RAG

In [ ]:
!pip install transformers datasets faiss-cpu sentence-transformers

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import torch
import numpy as np

In [ ]:
!pip install pyttsx3

In [ ]:
import pyttsx3

def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [ ]:
#loading models
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # fast & good
gpt2 = pipeline("text-generation", model="gpt2-medium")

In [ ]:
docs = [
    "Artificial Intelligence is the simulation of human intelligence in machines.",
    "Python is a high-level, interpreted programming language.",
    "The Earth revolves around the Sun in an elliptical orbit.",
    "Jarvis is a personal assistant AI system inspired by Iron Man."
]
doc_embeddings = embedder.encode(docs)

In [ ]:
import faiss

dim = doc_embeddings.shape[1]  #tells the embedding size(no. of columns)
index = faiss.IndexFlatL2(dim)
index.add(np.array(doc_embeddings))

In [ ]:
def get_relevant_doc(query):
    query_embedding = embedder.encode([query])
    D, I = index.search(np.array(query_embedding), k=1) #D is the distance/similarity score, I is the index of the closest document
    return docs[I[0][0]]

In [ ]:
#function to find relevant documents based on user query
def generate_with_context(query):
    context = get_relevant_doc(query)
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    output = gpt2(prompt, max_new_tokens=50, truncation=True, pad_token_id=50256)
    return output[0]['generated_text'] #returns the final generated text

In [ ]:
!pip install gTTS

In [ ]:
# ✅ gTTS for Text-to-Speech
from gtts import gTTS
from IPython.display import Audio, display
import os

def speak_text(text):
    tts = gTTS(text=text, lang='en')
    tts.save("jarvis_output.mp3")
    display(Audio("jarvis_output.mp3", autoplay=True))

In [ ]:
query = input("🎤 Type your question for Jarvis: ")
response = generate_with_context(query)
print("Jarvis says:", response)
speak_text(response)